In [1]:
# Extensions
%load_ext autoreload
%autoreload 2

In [14]:
# Imports
from word2vec import *
from core.search.query_paper_mag import paper_mag_multiquery
from core.search.query_info      import paper_info_mag_check_multiquery
from core.utils.entity_type      import Entity_type
from torch.utils.data            import DataLoader
import torch.optim as optim

DATA_LOC = 'test.dat'

In [3]:
# Get paper information

# Conf id
conf_id = 1127352206 #PLDI

# Generate papers for the conference
papers = paper_mag_multiquery(Entity_type.CONF, [conf_id])

print("Papers for conference:", len(papers))

# Import paper information here!
paper_informations = paper_info_mag_check_multiquery(papers)

return statue: 403
{'Ocp-Apim-Subscription-Key': '004967a3daac4bf683e16ca2a459de07'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'ce7734c51f504dcaa1dd89cb88fa153d'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': '9c0c0c5e7758484b9ed8092034eac796'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be replenished later." } }\n\t\t        \t\t\t\t  '
return statue: 403
{'Ocp-Apim-Subscription-Key': 'aeea5a7bb8fd4d62af6c9c08b26521dc'}
ERROR: problem with the request.
b'\n{ "error": { "code": "QuotaExceeded", "message": "Out of call volume quota. Quota will be reple

In [4]:
# Preprocess
preprocess = Preprocess(paper_informations)
preprocess.gen_pairs()
preprocess.export_pairs(DATA_LOC)

In [36]:
# Get context corpus
corpus = ContextCorpus(DATA_LOC)
print(len(corpus))

85840


In [69]:
# SGNS model
embedding_size = len(preprocess.index)
embedding_dim  = 128
sgns = SGNS(embedding_size, embedding_dim)
print(sgns)

SGNS(
  (w_embedding): Embedding(2277, 128)
  (c_embedding): Embedding(2277, 128)
)


In [72]:
from tqdm import tqdm

BATCH_SIZE = 2000
ITERATIONS = 10
NUM_NEG    = 1
LEARN_RATE = 0.1 #0.025

optimizer = optim.Adagrad(sgns.parameters(), lr = LEARN_RATE)

num_batches = int(ITERATIONS * len(corpus) / BATCH_SIZE)

pos_dataloader = DataLoader(corpus, batch_size=BATCH_SIZE, shuffle=True)
neg_dataloader = DataLoader(corpus, batch_size=NUM_NEG, shuffle=True)

pos_iter = iter(pos_dataloader)
neg_iter = iter(neg_dataloader)

progress_bar = tqdm(range(1, num_batches + 1))

for epoch in progress_bar:
    #pos_w, pos_c = pos_iter.next()
    #neg_w, neg_c = neg_iter.next()

    try:
        pos_w, pos_c = pos_iter.next()
    except StopIteration:
        pos_iter = iter(pos_dataloader)
        pos_w, pos_c = pos_iter.next()

    try:
        neg_w, neg_c = neg_iter.next()
    except StopIteration:
        neg_iter = iter(neg_dataloader)
        neg_w, neg_c = neg_iter.next()

    optimizer.zero_grad()
    loss = sgns.forward(pos_w, pos_c, neg_w, neg_c)
    loss.backward()
    optimizer.step()
    
    progress_bar.set_description("Loss: %0.8f, lr: %0.6f" % (loss.data.item(), optimizer.param_groups[0]['lr']))
    print("Loss: %0.8f, lr: %0.6f" % (loss.data.item(), optimizer.param_groups[0]['lr']))














  0%|          | 0/429 [00:00<?, ?it/s]












Loss: 8607.78710938, lr: 0.100000:   0%|          | 0/429 [00:00<?, ?it/s]












Loss: 8607.78710938, lr: 0.100000:   0%|          | 1/429 [00:00<00:43,  9.82it/s]












Loss: 7979.07226562, lr: 0.100000:   0%|          | 1/429 [00:00<01:19,  5.41it/s]












Loss: 7252.45410156, lr: 0.100000:   0%|          | 1/429 [00:00<01:43,  4.14it/s]












Loss: 7252.45410156, lr: 0.100000:   1%|          | 3/429 [00:00<00:34, 12.22it/s]

Loss: 8607.78710938, lr: 0.100000
Loss: 7979.07226562, lr: 0.100000
Loss: 7252.45410156, lr: 0.100000















Loss: 6376.69677734, lr: 0.100000:   1%|          | 3/429 [00:00<00:47,  9.06it/s]












Loss: 6056.38720703, lr: 0.100000:   1%|          | 3/429 [00:00<00:54,  7.79it/s]












Loss: 6056.38720703, lr: 0.100000:   1%|          | 5/429 [00:00<00:32, 12.86it/s]












Loss: 5702.67333984, lr: 0.100000:   1%|          | 5/429 [00:00<00:37, 11.39it/s]












Loss: 5219.16992188, lr: 0.100000:   1%|          | 5/429 [00:00<00:43,  9.83it/s]












Loss: 5219.16992188, lr: 0.100000:   2%|▏         | 7/429 [00:00<00:30, 13.65it/s]

Loss: 6376.69677734, lr: 0.100000
Loss: 6056.38720703, lr: 0.100000
Loss: 5702.67333984, lr: 0.100000
Loss: 5219.16992188, lr: 0.100000















Loss: 4910.75244141, lr: 0.100000:   2%|▏         | 7/429 [00:00<00:35, 11.91it/s]












Loss: 4745.81152344, lr: 0.100000:   2%|▏         | 7/429 [00:00<00:38, 10.87it/s]












Loss: 4745.81152344, lr: 0.100000:   2%|▏         | 9/429 [00:00<00:30, 13.89it/s]












Loss: 4342.19287109, lr: 0.100000:   2%|▏         | 9/429 [00:00<00:32, 12.81it/s]

...

Loss: 18.26308632, lr: 0.100000
Loss: 15.01538277, lr: 0.100000















Loss: 17.36529922, lr: 0.100000:  98%|█████████▊| 422/429 [00:28<00:00, 14.73it/s]












Loss: 17.36529922, lr: 0.100000:  99%|█████████▉| 424/429 [00:28<00:00, 14.80it/s]












Loss: 18.52570724, lr: 0.100000:  99%|█████████▉| 424/429 [00:28<00:00, 14.75it/s]












Loss: 13.16580200, lr: 0.100000:  99%|█████████▉| 424/429 [00:28<00:00, 14.73it/s]












Loss: 13.16580200, lr: 0.100000:  99%|█████████▉| 426/429 [00:28<00:00, 14.80it/s]












Loss: 8.11974335, lr: 0.100000:  99%|█████████▉| 426/429 [00:28<00:00, 14.77it/s] 

Loss: 17.36529922, lr: 0.100000
Loss: 18.52570724, lr: 0.100000
Loss: 13.16580200, lr: 0.100000















Loss: 8.31846237, lr: 0.100000:  99%|█████████▉| 426/429 [00:28<00:00, 14.75it/s]












Loss: 8.31846237, lr: 0.100000: 100%|█████████▉| 428/429 [00:28<00:00, 14.81it/s]












Loss: 10.92595482, lr: 0.100000: 100%|█████████▉| 428/429 [00:28<00:00, 14.79it/s]












Loss: 10.92595482, lr: 0.100000: 100%|██████████| 429/429 [00:28<00:00, 14.83it/s]

Loss: 8.11974335, lr: 0.100000
Loss: 8.31846237, lr: 0.100000
Loss: 10.92595482, lr: 0.100000
